# Librairies

In [2]:
import os
import json

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.utils import Sequence, to_categorical

print(tf.test.is_gpu_available())

min_frame = 5

True


# Data

In [2]:
list_IDs = {}
with open("data/dataset.json", "r") as file:
    list_IDs = json.load(file)

In [4]:
class NTUSequence(Sequence):
    
    def __init__(self, list_IDs, path, batch_size=32, one_hot=True, min_frame=min_frame):
        # Removed unused vars (self.x, max_frame)
        self.x = list_IDs
        self.batch_size = batch_size
        self.path = path
        self.min_frame = min_frame
        if one_hot:
            self.one_hot_encode()

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx, frame):
        #idx: index of the data
        # frame: starting frame
        
        # Should be the same as normal
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        #Yeah now we're having fun
        batch_y = np.array([self.x[ID] for ID in batch_x])
        
        #list of batch_x files 
        F = [np.load(self.path + ID + '.npy')[:, :, :, np.newaxis] for ID in batch_x]
        
        if not(self.min_frame is None):
            # list of batch_x arrays of min_frames frames
            X = np.stack(
                [x[:min_frame] if x.shape[0] >= min_frame else np.pad(x, ((0, min_frame-x.shape[0]),(0,0),(0,0),(0,0)), "constant") for x in F]
            )
            Y = np.stack(
                [y[min_frame]] for y in F
            )
        
        return X, batch_y

#     def one_hot_encode(self):
#         labels = [val for val in self.y.values()]
#         ids = [key for key in self.y.keys()]
#         labels = to_categorical(labels)
#         self.y = {ID: label for ID, label in zip(ids, labels)}

In [5]:
trainset = NTUSequence(list_IDs["train"], path="D:/PFE/train/")
testset = NTUSequence(list_IDs["validation"], path="D:/PFE/test/")

TypeError: __init__() missing 1 required positional argument: 'labels'

# Architecture

In [3]:
convnet = models.Sequential([
    
    layers.Conv2D(64, 3, activation="relu", padding="same", input_shape=(25, 25, 1)),
    layers.Conv2D(64, 3, activation="relu", padding="same"),
    layers.Conv2D(64, 3, activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
                        
    layers.Conv2D(512, 3, activation="relu", padding="same"),
    layers.Conv2D(512, 3, activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten()
                        ])

In [4]:
convnet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 25, 25, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 64)        36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 128)         0

In [6]:
predictor = models.Sequential([
    layers.Input((None, 25, 25, 1)),
    layers.TimeDistributed(convnet),
    
    layers.LSTM(9, activation="relu"),
    layers.Reshape((3,3,1)),
    
    layers.Conv2DTranspose(1, 3, strides=2, activation="relu", padding="same"),
    layers.Conv2DTranspose(1, 3, strides=2, activation="relu", padding="same"),
    layers.Conv2DTranspose(1, 3, strides=2, activation="relu"),
    
    layers.Reshape((25,25))
])

In [7]:
predictor.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, None, 512)         4721152   
_________________________________________________________________
lstm (LSTM)                  (None, 9)                 18792     
_________________________________________________________________
reshape (Reshape)            (None, 3, 3, 1)           0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 6, 6, 1)           10        
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 12, 12, 1)         10        
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 25, 25, 1)         10        
_________________________________________________________________
reshape_1 (Reshape)          (None, 25, 25)           